In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack



In [19]:
df = pd.read_csv('merged_cleaned.csv',
                 parse_dates= [0])

In [3]:
df.head()

,Gesamt (Netzlast) [MWh] Berechnete Auflösungen,Deutschland/Luxemburg [€/MWh] Originalauflösungen,Biomasse [MWh] Berechnete Auflösungen,Wasserkraft [MWh] Berechnete Auflösungen,Wind Offshore [MWh] Berechnete Auflösungen,Wind Onshore [MWh] Berechnete Auflösungen,Photovoltaik [MWh] Berechnete Auflösungen,Sonstige Erneuerbare [MWh] Berechnete Auflösungen,Kernenergie [MWh] Berechnete Auflösungen,Braunkohle [MWh] Berechnete Auflösungen,...,date_offset,hour_sin,hour_cos,dayofweek_sin,dayofweek_cos,month_sin,month_cos,dayofyear_sin,dayofyear_cos,is_workday
Date,,,,,,,,,,,,,,,,,,,,,
2018-11-01 00:00:00,48879.50,40.86,4754.75,1583.00,2721.00,11628.50,0.0,153.00,7933.0,16108.75,...,781,0.000000,1.000000,0.433884,-0.900969,-0.5,0.866025,-0.858764,0.512371,1
2018-11-01 01:00:00,47312.50,41.10,4741.25,1587.50,2678.00,11029.00,0.0,153.00,7933.5,16119.00,...,781,0.258819,0.965926,0.433884,-0.900969,-0.5,0.866025,-0.858764,0.512371,1
2018-11-01 02:00:00,46093.75,40.59,4733.25,1567.50,2836.75,11025.00,0.0,153.00,7927.0,16111.00,...,781,0.500000,0.866025,0.433884,-0.900969,-0.5,0.866025,-0.858764,0.512371,1
2018-11-01 03:00:00,46710.50,40.09,4725.25,1550.00,2968.50,10480.25,0.0,152.50,7930.5,16134.75,...,781,0.707107,0.707107,0.433884,-0.900969,-0.5,0.866025,-0.858764,0.512371,1
2018-11-01 04:00:00,47624.25,39.11,4715.00,1535.75,2765.25,10871.25,0.0,152.75,7935.5,16120.50,...,781,0.866025,0.500000,0.433884,-0.900969,-0.5,0.866025,-0.858764,0.512371,1


In [4]:
df.Holiday.unique()

array(['Not a Holiday', 'Christmas Day', 'Second Christmas Day',
       'New year', 'Good Friday', 'Easter Monday', 'Labour Day',
       'Ascension Thursday', 'Whit Monday', 'Day of German Unity'],
      dtype=object)

In [20]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
numerical_columns = df.select_dtypes(include=['number']).columns.drop('Gesamt (Netzlast) [MWh] Berechnete Auflösungen')

X = df.drop('Gesamt (Netzlast) [MWh] Berechnete Auflösungen', axis=1)
y = df[['Date', 'Gesamt (Netzlast) [MWh] Berechnete Auflösungen']]

# Separate categorical and numerical features
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

In [6]:
# Use OneHotEncoder with sparse output
encoder = OneHotEncoder(sparse_output=True, drop='first')
X_categorical_encoded = encoder.fit_transform(X_categorical)

# Combine numerical and sparse categorical features
X_encoded = hstack([X_numerical, X_categorical_encoded], format='csr')

In [7]:
# Train the model with multicore processing
model = RandomForestRegressor(random_state=42, n_jobs=-1)
model.fit(X_encoded, y)




RandomForestRegressor(n_jobs=-1, random_state=42)

In [13]:
# Retrieve feature names from OneHotEncoder and numerical columns
categorical_feature_names = encoder.get_feature_names_out(categorical_columns)
feature_names = list(numerical_columns) + list(categorical_feature_names)

# Get feature importances
importances = model.feature_importances_

# Create a DataFrame for better visualization
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

In [14]:
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances.head(50))  # Top 10 most important features


                                                 Feature  Importance
13                                                  hour    0.406127
28                                         dayofyear_cos    0.086600
14                                             dayofweek    0.086155
23                                         dayofweek_sin    0.066129
29                                            is_workday    0.065101
22                                              hour_cos    0.050592
20                                           date_offset    0.039825
18                                             dayofyear    0.029823
7               Kernenergie [MWh] Berechnete Auflösungen    0.028442
9                Steinkohle [MWh] Berechnete Auflösungen    0.024078
36                                 Holiday_Not a Holiday    0.023036
21                                              hour_sin    0.018790
4              Wind Onshore [MWh] Berechnete Auflösungen    0.010746
6      Sonstige Erneuerbare [MWh] 

In [10]:
threshold = 0.01  # Example threshold
selected_features = feature_importances[feature_importances['Importance'] > threshold]['Feature']
print(f"Number of selected features: {len(selected_features)}")


Number of selected features: 13


In [17]:
y.head()

Date
2018-11-01 00:00:00    48879.50
2018-11-01 01:00:00    47312.50
2018-11-01 02:00:00    46093.75
2018-11-01 03:00:00    46710.50
2018-11-01 04:00:00    47624.25
Name: Gesamt (Netzlast) [MWh] Berechnete Auflösungen, dtype: float64

In [11]:
selected_feature_indices = feature_importances[feature_importances['Importance'] > threshold].index
X_selected = X_encoded[:, selected_feature_indices]  # Keeps only important features


In [21]:
# Convert sparse matrix to a dense DataFrame with selected feature names
X_selected_dense = pd.DataFrame(X_selected.toarray(), columns=selected_features)

# Combine the features and target variable
filtered_dataset = pd.concat([X_selected_dense, y.reset_index(drop=True)], axis=1)

# Save to CSV
filtered_dataset.to_csv('selected_features.csv', index=False)
print("Filtered dataset saved as 'selected_features.csv'.")

Filtered dataset saved as 'selected_features.csv'.
